# Objective :-
Here we will replace the Nan values in the 'Target' column of 'train.csv' with the recreated targets present here https://www.kaggle.com/swaralipibose/target-recreation-but-its-99-percent-accurate so that we can gather up more data it has also been discussed here https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/293654 which has proposed this method

# Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
df=pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
df.head()

In [ ]:
details=pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
details.head()

# Percentage of Nan values

In [ ]:
import numpy as np

for i in range(14):
 
    dfcrop=df[df['Asset_ID']==i]
    print('Percentage of values not nan',(1-(np.sum((dfcrop['Target'].isnull()).astype(int))/dfcrop.shape[0]))*100)
  
    


# Recreate Targets
note 
if you are confused ehy R-1 is being used for creating targets , if you have seen the recreation notebook [linked above]then you might no that target=np.exp(R)-1
R=np.log(Price after 16 min/Price after 1 min) 
so np.exp is just undoing np.log so it can be written as


(Price after 16min/Price after 1min)- 1

In [ ]:
from tqdm import tqdm
def compute(df):
    
    R=list()
    c=list(df['Close'])
    for i in range(df.shape[0]):
        future=c[min([i+16,df.shape[0]-1])]
        past=c[min([i+1,df.shape[0]-1])]
        R.append(future/past)
    R=np.array(R)

    df['pred']=R-1
    return df
crops=list()
for a in tqdm(list(range(14))):
    
    crops.append(compute(df[df['Asset_ID']==a]))


In [ ]:

conc=pd.concat(crops)

In [ ]:
conc.head()

In [ ]:
conc=conc.reset_index()

In [ ]:
j=np.array(list(conc['Target'].isnull()))
new_targets=np.where(j,conc['pred'],conc['Target'])

In [ ]:
conc['Target']=new_targets
conc=conc.drop(columns=['pred'],axis=1)
conc.head()

# No more nan values 

In [ ]:
import numpy as np

for i in range(14):
    dfcrop=conc[conc['Asset_ID']==i]
    print('Percentage of values not nan',(1-(np.sum((dfcrop['Target'].isnull()).astype(int))/dfcrop.shape[0]))*100)
  
    


# Save new train.h5 file h5 is way faster

In [ ]:
conc.to_hdf('train.h5',key='conc', mode='w',index=False)

In [ ]:
del conc,crops

# Load your saved h5 file

In [ ]:
df=pd.read_hdf('train.h5')
df=df.drop(columns=['index'])
df.head()

In [ ]:
df.shape

# Thank you